# PART 2: Issue a VC to the Manufacturer Agents

In [1]:
%%javascript
document.title='Authority Agent'

<IPython.core.display.Javascript object>

#### Imports

In [2]:
from aries_cloudcontroller import AriesAgentController
from libs.connection_service import ConnectionService
import libs.functions as fnc
import os
from pprintpp import pprint
from termcolor import colored

#### Get schema and cred def identifiers defined in `00_init_authority_as_issuingAuthority.ipynb`

In [3]:
# Copy an updated version if necessary
identifiers = fnc.get_identifiers() 

# Get city schema identifiers
schema_city_id = identifiers["city_schema_identifiers"]["schema_id"]
cred_def_city_id = identifiers["city_schema_identifiers"]["cred_def"]

# Get manufacturer schema identifiers
schema_manufacturer_id = identifiers["manufacturer_schema_identifiers"]["schema_id"]
cred_def_manufacturer_id = identifiers["manufacturer_schema_identifiers"]["cred_def"]

#### Get relevant variables

In [4]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

## 1 – Init Aries Agent, Webhook Server and register Event Listeners for webhook server
#### 1.1 – Init aries agent and Start webhook server
Start a webhook server to be able to communicate with other agents

In [5]:
# Setup
print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")
agent_controller = AriesAgentController(admin_url,api_key)

# Listen on webhook server
await agent_controller.init_webhook_server(webhook_host, webhook_port)
print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Initialising a controller with admin api at http://authority-agent:3021 and an api key of adminApiKey
Listening for webhooks from agent at http://0.0.0.0:3010


#### 1.2 – Define listeners
Define listeners that are triggered when something happens on the webhook server

#### 1.3 – Register listeners with `agent_controller`

In [6]:
connections = ConnectionService(agent_controller)

Initiate ConnectionService:
* Defines and registers agent listeners:  ['connections', 'prover_proof', 'verifier_proof', 'issue_credential', 'basicmessages']
* Stores initiated connections
* Allows to easily create and accept connection invitations
* Facilitates process of issuing and verifying verifiable credentials


In [21]:
# Setup variables
auto_accept = "true"
alias = None
label = "City Agent <> Authority Agent" 

connection_id = connections.wrapper_receive_connection_invitation(alias, auto_accept, label)

Please enter invitation received by external agent:  {     '@id': '4747415c-1935-408c-a794-6a73bb6b309c',     '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',     'label': 'Manufacturer1',     'recipientKeys': ['FWj2XhxFNke6DFusyZfkirEv7rXYRSqMCdpMJNEMMNEv'],     'serviceEndpoint': 'http://bb082e6e29a4.ngrok.io', }



---------------------------------------------------------------------
Connection Webhook Event Received
Connection ID :  8d2736f5-476b-4a76-9626-6aa535054e12
State :  invitation (invitation-received)
Routing State : none
Connection with :  Manufacturer1
Their Role :  inviter
---------------------------------------------------------------------

---------------------------------------------------------------------
Connection Webhook Event Received
Connection ID :  8d2736f5-476b-4a76-9626-6aa535054e12
State :  request (request-sent)
Routing State : none
Connection with :  Manufacturer1
Their Role :  inviter
---------------------------------------------------------------------


**BREAK POINT:** Please switch to agent `Manufacturer1`, open `01_hold_VC.ipynb`, and start executing the notebook

---

## 2 – Establish connection with `Manufacturer1` agent
Establish connection with the`Manufacturer1` agent. In this scenario, the `Manufacturer1` sends an invitation to connect with the Authority. 

### 2.1 – Receive invitation from `Manufacturer1` agent
Copy the invitation from Step 2.2 in the City's `01_hold_VC.ipynb` notebook into the following cell. 

### 2.2 – Accept invitation

**BREAK POINT:** Please proceed to step 2.3 in the `01_hold_VC.ipynb` notebook of the `Manufacturer1` agent

---

## 3 – Get info from `Manufacturer1` agent and issue a VC
### 3.1 – Greet `Manufacturer1` to open connection in this cell
The response with the relevant City information will be displayed in the following cell.

In [ ]:
connections.send_message(connection_id, "testing connection")

In [ ]:
basic_message = "Hello Manufacturer1. Plase provide me with the following information: manufacturerName, manufacturerCountry, ManufacturerCity."
await agent_controller.messaging.send_message(connection_id, basic_message)

### 3.2 – Populate VC Attributes

Define the values that will be issued in the VC. The attributes match the attributes defined in the city-agent-scheme.

In [16]:
# @todo: see if you can automate the filling out of this (e.g., by accessing the scheme or similar)
values_m1 = {"manufacturerName": "M1", "manufacturerCountry": "DE", "manufacturerCity": "Munich"}

credential_attributes = [
    {"name": "manufacturerName", "value": values_m1["manufacturerName"]},
    {"name": "manufacturerCity", "value": values_m1["manufacturerCity"]},
    {"name": "manufacturerCountry", "value": values_m1["manufacturerCountry"]},
    {"name": "isManufacturer", "value": "TRUE"}
]
print(credential_attributes)

[{'name': 'manufacturerName', 'value': 'M1'}, {'name': 'manufacturerCity', 'value': 'Munich'}, {'name': 'manufacturerCountry', 'value': 'DE'}, {'name': 'isManufacturer', 'value': 'TRUE'}]


In [14]:
await agent_controller.schema.get_by_id(schema_manufacturer_id)

{'schema': {'ver': '1.0',
  'id': 'XnKQj3iXVNpBBtFcArtxw2:2:certify-manufacturer:0.0.1',
  'name': 'certify-manufacturer',
  'version': '0.0.1',
  'attrNames': ['manufacturerCountry',
   'manufacturerName',
   'isManufacturer',
   'manufacturerCity'],
  'seqNo': 248534}}

### 3.3 – Offer VC to `Manufacturer1` agent

In [19]:
# MAKE VC ZKP-able! SEE https://github.com/hyperledger/aries-cloudagent-python/blob/main/JsonLdCredentials.md
comment = "Issuing VC that Manufacturer1 is a manufacturer"
auto_remove = True
trace = False

# Cred_def_id must identify a definition to which your agent has corresponding private issuing key.
send_cred_response = await agent_controller.issuer.send_credential(connection_id, schema_manufacturer_id, cred_def_manufacturer_id, credential_attributes, comment, auto_remove, trace)


---------------------------------------------------------------------
Handle Issue Credential Webhook
Connection ID : 8d2b36e8-3fbc-4044-b702-d4d852c0e1ac
Credential exchange ID : f224e64c-73dd-4aaf-b545-472c43f64ead
Agent Protocol Role :  issuer
Protocol State :  offer_sent
---------------------------------------------------------------------
Offering : 
 [{'name': 'manufacturerName', 'value': 'M1'}, {'name': 'manufacturerCity', 'value': 'Munich'}, {'name': 'manufacturerCountry', 'value': 'DE'}, {'name': 'isManufacturer', 'value': 'TRUE'}]


**BREAK POINT:** Proceed to `Manufacturer1` Agent, Step 3.2 

---

## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [ ]:
await agent_controller.terminate()